### Dynamic model

**동적 모델**(Dynamic model)은 에이전트 실행 중에 현재 **상태**(state)와 **컨텍스트**(context)를 기반으로 선택되는 모델을 의미합니다.</br>
이는 정적 모델(Static model)과 달리, 에이전트가 실행되는 런타임(runtime) 환경에서 특정 조건에 따라 사용할 모델을 유연하게 변경할 수 있게 합니다.

#### 🌟 동적 모델의 특징 및 사용 목적

+ 선택 시점: 런타임에 결정됩니다.
+ 활용: 복잡한 라우팅 로직(routing logic) 구현 및 비용 최적화를 가능하게 합니다.
+ 구현 방법: 주로 @wrap_model_call 데코레이터를 사용하여 미들웨어(middleware) 형태로 구현합니다. 이 미들웨어가 모델 호출 요청(request)을 수정하여 사용할 모델을 지정합니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

basic_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
advanced_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)

agent = create_agent(
    model=basic_model,  # Default model
    # tools=tools,
    middleware=[dynamic_model_selection]
)

In [4]:
# Run the agent
result = agent.invoke(
    {"messages": [{"role": "user", "content": "질문"}]}
)

In [5]:
result

{'messages': [HumanMessage(content='질문', additional_kwargs={}, response_metadata={}, id='f1dfa2a8-ddc8-49b1-bb68-98e02457aeb7'),
  AIMessage(content='무엇을 도와드릴까요? 질문해주세요.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--c161fa7f-25bc-4ee7-bcfa-a7ced0258d41-0', usage_metadata={'input_tokens': 3, 'output_tokens': 10, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})]}

In [13]:
result["messages"][1].response_metadata["model_name"]

'gemini-2.5-flash-lite'

In [16]:
# 최소 11개의 메시지(HumanMessage 또는 AIMessage)를 포함하는 history 준비
from langchain_core.messages import HumanMessage, AIMessage

# HumanMessage와 AIMessage를 번갈아 가며 11개 생성 (예시)
long_history = [
    HumanMessage(content=f"질문 {i+1}") if i % 2 == 0 else AIMessage(content=f"답변 {i+1}") 
    for i in range(11) # 메시지 개수: 11개 (11 > 10 조건 충족)
]

In [17]:
long_history

[HumanMessage(content='질문 1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 6', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 7', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 8', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 9', additional_kwargs={}, response_metadata={}),
 AIMessage(content='답변 10', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문 11', additional_kwargs={}, response_metadata={})]

In [18]:
from langchain_core.messages import HumanMessage

# 이전 대화 기록 (11개의 메시지)

# 1. 최종 사용자 질문을 HumanMessage로 만듭니다.
final_message = HumanMessage(content="이 긴 대화의 맥락에서 다음 질문에 답해줘.")

# 2. 전체 대화 기록을 하나의 'messages' 리스트로 합칩니다.
full_conversation = long_history + [final_message]

# 3. 에이전트에게 'messages' 키를 사용하여 전체 메시지 리스트를 전달합니다.
#    (LangChain Agents/Graph에서 대화 기록을 처리하는 가장 표준적인 방법)
result = agent.invoke(
    {
        "messages": full_conversation # 모든 메시지를 이 키에 통합하여 전달
    }
)

In [19]:
result

{'messages': [HumanMessage(content='질문 1', additional_kwargs={}, response_metadata={}, id='f3ec3b20-775a-4843-9fd3-aba251ae7fd6'),
  AIMessage(content='답변 2', additional_kwargs={}, response_metadata={}, id='e6c56713-ede0-4f66-bb42-ff53543968b9'),
  HumanMessage(content='질문 3', additional_kwargs={}, response_metadata={}, id='b2373575-5622-4137-9de9-5f5ffc58910d'),
  AIMessage(content='답변 4', additional_kwargs={}, response_metadata={}, id='dd0c70d4-f3aa-4357-be7b-2cb1fb1c5a2e'),
  HumanMessage(content='질문 5', additional_kwargs={}, response_metadata={}, id='74810fd3-4005-4069-a723-64af1c7918cc'),
  AIMessage(content='답변 6', additional_kwargs={}, response_metadata={}, id='ca43a440-9031-41e3-8187-19071c1df5a8'),
  HumanMessage(content='질문 7', additional_kwargs={}, response_metadata={}, id='7b21d7b0-feac-454a-9a96-195244ee4bc8'),
  AIMessage(content='답변 8', additional_kwargs={}, response_metadata={}, id='7c98f3ff-3e5c-492a-8f2a-667cb31f94fd'),
  HumanMessage(content='질문 9', additional_kwargs

In [22]:
result["messages"][-1]

AIMessage(content='네, 질문 11이 무엇인지 알려주시면 이 긴 대화의 맥락을 고려하여 답변해 드리겠습니다. 질문을 말씀해주세요!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--9c4c936a-6d45-4340-b5f6-a8295758313a-0', usage_metadata={'input_tokens': 73, 'output_tokens': 108, 'total_tokens': 181, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 76}})

In [24]:
result["messages"][-1].response_metadata["model_name"]

'gemini-2.5-flash'